
# Moments of Inertia

In [ ]:
# import smilPython 
import smilPython as sp

# functions using matplotlib to display images on jupyter notebooks
from smilPyGoodies import *

import numpy as np

## Let's begin


In [ ]:
from math import *
 
# Load an image
imIn = sp.Image("images/tools.png")

imThr = sp.Image(imIn)
sp.topHat(imIn, imThr, sp.hSE(20))
sp.threshold(imThr, imThr)

imLbl = sp.Image(imIn, "UINT16")
sp.label(imThr, imLbl)

ImShow([imIn, imThr, imLbl], fakeColor = [False, False, True], titles = ["imIn", "imThr", "imLbl"])

In [ ]:
def fitRectangle(mat):
    m00, m10, m01, m11, m20, m02 = mat

    if m00 == 0:
      return 0, 0, 0, 0, 0

    # COM
    xc = int (m10 / m00)
    yc = int (m01 / m00)

    # centered matrix (central moments)
    u00 = m00
    u20 = m20 - m10**2 / m00
    u02 = m02 - m01**2 / m00
    u11 = m11 - m10 * m01 / m00

    # eigen values
    delta = 4 * u11**2 + (u20 - u02)**2
    I1 = (u20 + u02 + sqrt(delta)) / 2
    I2 = (u20 + u02 - sqrt(delta)) / 2

    theta = 0.5 * atan2(-2 * u11, (u20 - u02))

    # Equivalent rectangle
    # I1 = a**2 * S / 12, I2 = b**2 * S / 12
    a = int (sqrt(12 * I1 / u00))
    b = int (sqrt(12 * I2 / u00))

    return xc, yc, a, b, theta

In [ ]:
# Compute Blobs
blobs = sp.computeBlobs(imLbl)

# Compute Inertia Matrices
mats   = sp.measImageBlobsMoments(imIn, blobs)
bboxes = sp.measBoundBoxes(imLbl)

In [ ]:
imDraw = sp.Image(imIn)

print("{0:5s} | {1:5s} | {2:5s} | {3:7s}".format("Label", "  A", "  B", "Theta"))
for b in blobs.keys():
    mat = xc, yc, A, B, theta = fitRectangle(mats[b])
    print("{0:5d} | {1:5d} | {2:5d} | {3:7.4f}".format(b, A, B, theta))
    dx = A / 2 * cos(pi - theta)
    dy = A / 2 * sin(pi - theta)
    sp.drawLine(imDraw, int(xc - dx), int(yc - dy), int(xc + dx), int(yc + dy), b)
    dx = B / 2 * sin(theta)
    dy = B / 2 * cos(theta)
    sp.drawLine(imDraw, int(xc - dx), int(yc - dy), int(xc + dx), int(yc + dy), b)

In [ ]:
ImShow([imIn, imDraw], fakeColor = [False, True], titles = ["Original", "Inertia"])